# 3.7.3: Baseball

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

## The Manny Ramirez Problem

Manny Ramirez is a former member of the Boston Red Sox (an American
baseball team) who was notorious for his relaxed attitude and taste for practical jokes. Our objective in this chapter is to solve the following Manny-inspired problem:  What is the minimum effort required to hit a home run in Fenway Park?

<br>

Fenway Park is a baseball stadium in Boston, Massachusetts. One of its
most famous features is the "Green Monster", which is a wall in left
field that is unusually close to home plate, only 310 feet away. To
compensate for the short distance, the wall is unusually high, at 37
feet.

Here's the functions we've created:

In [ ]:
# @title
import pandas as pd
import numpy as np

def angle_to_x(mag,angle):
    theta = np.deg2rad(angle)
    x = mag * np.cos(theta)
    return x

def angle_to_y(mag,angle):
    theta = np.deg2rad(angle)
    y = mag * np.sin(theta)
    return y

def unit_vec(V, mag):
    return  V/mag

def drag_force(V, system):
    rho, C_d, area = system['rho'], system['C_d'], system['area']

    # Find the magnitude and direction of the velocity
    vel_mag = np.sqrt(V.x**2 + V.y**2)
    if vel_mag != 0:
        dir = unit_vec(V, vel_mag)
    else:
        dir = pd.Series(dict(x = 0, y = 0), dtype = float)

    # Find the magnitude of the drag force
    drag_mag = rho * vel_mag**2 * C_d * area * (1/2)

    # Define the direction of the force as opposite that of the  velocity
    # Notice that "dir" is a vector, so f_drag is vector too
    f_drag = drag_mag * -dir

    return f_drag

def change_func(t, state, system):
    x, y, vx, vy = state
    mass, g, dt = system['mass'], system['g'], system['dt']

    V = pd.Series(dict(x=vx, y=vy),dtype=float)
    a_drag = drag_force(V, system) / mass

    # Acceleration has to be defined as a vector too
    a_grav = pd.Series(dict(x=0,y=-g),dtype=float)

    A = a_grav + a_drag

    x = x + V.x*dt
    y = y + V.y*dt
    vx = vx + A.x*dt
    vy = vy + A.y*dt

    return pd.Series(dict(x=x, y=y, vx=vx, vy=vy))

### Creating a system

We are going to be using several "black box" algorithms (`minimize_scalar` and (`root_scalar`), so we need to make sure everything we need to run a simulation and create a state is packed into our System object.   So that object contains:

 * a few old variables here (`angle` and `speed`) that were not in system before.
 * two new parameters (`wall_dist` and `wall_h`, in meters) that are necessary for this particular problem.
 * the function `change_func`, which we'll need to call in our `run_simulation`

 Here's the system:

In [ ]:
import pandas as pd

# Initial state variables
x_init = 0          # m
y_init = 1          # m
angle = 45          # degree
speed = 40          # m / s

# System parameters
mass = 0.145        # kg
diam = 0.073        # m
C_d = 0.33          # dimensionless
rho = 1.2           # kg/m**3
g = 9.8             # m/s**2
feet_to_m = 0.3048
wall_dist = 310*feet_to_m   # m (94.48 m)
wall_h = 37*feet_to_m     # m (11.28 m)

# Simulation parameters
t_end = 10          # s
dt = 0.01           # s

system = dict(x_init=x_init, y_init=y_init, angle=angle, speed=speed,
              mass=mass, area = np.pi * (diam/2)**2, C_d=C_d, rho=rho,
              g=g, wall_dist = wall_dist, wall_h=wall_h,
              t_end=t_end, dt=dt, change_func=change_func)

The answer we want is the minimum speed at which a ball can leave home plate and still go over the Green Monster. We'll proceed in the
following steps:

<br>

1.  For a given speed, we'll find the optimal *launch angle*, that is, the angle the ball should leave home plate to maximize its height when it reaches the wall.

2.  Then we'll find the minimum speed that clears the wall, given that it has the optimal launch angle.



### Part 1

As a first step, revise `run_simulation` so that the simulation stops when the ball reaches the wall at `wall_distance`.
Test your function with the initial conditions (it should return `wall_distance`):

In [ ]:
# Here's the original run_simulation
def run_simulation(system, state, change_func):
    # Define the time steps
    t_array = np.arange(0, system['t_end']+1, system['dt'])
    n = len(t_array)

    # Set up a DataFrame to store the our state variables
    results = pd.DataFrame(index=t_array, columns=state.index,
                        dtype=np.float64)
    results.iloc[0] = state

    for i in range(n-1):
        t = t_array[i]
        state = change_func(t, state, system)
        results.iloc[i+1] = state
        # Test to see if the penny has hit the ground
        if state.y <= 0.0:
            results = results.dropna()
            return results

    return results

In [ ]:
# Create a state and run a simulation make sure the final value of x is wall_distance


### Part 2

Now we need a function that takes a launch angle and returns the height of the baseball when it reaches the wall.  'height_func` should:
* take a launch angle and a system as its arguments
* unpack the variables necessary to make a new State object (including `change_func`)
* create a State object with the new launch angle
* run a simulation with the system and the new state
* store the height of the baseball when it reaches the wall (that is, at the end of the simulation)
* return the negative height of the baseball when it reaches the wall (negative so that we can minimize this function)

<br>

Test your function with a 45 degree angle and normal parameters: you should find that the height function returns a value of about -6.9 m (Did you remember to make the return value negative so we could "minimize" it? 😀)

In [ ]:
# Define height function
# If you are unsure how to do this, follow the steps listed above
# one by one!
def height_func(angle,system):


In [ ]:
# Test height_function
height_func(45,system)

If your function correctly returns -6.9 m, this means that when the ball reached the wall it was 6.9 m above the ground: not high enough to get over the Green Monster!

### Part 3

We now have a function that will tell us the height of the ball when it reaches the wall.  If we use this function in `minimize_scalar`, we can find the angle that will maximize this height.  Is it higher or lower than the angle that maximizes range?

<br>

The call to `minimize_scalar` includes an argument for `tol`.  This is a tolerance: this tells the algorithm how accurate the final answer needs to be.  A low tolerance (0.00001 is not unusual) means the simulation will be accurate but will take longer.  In this case, our tolerance is pretty high: we're willing to trade some accuracy for quicker simulations.

In [ ]:
# Find the optimal angle
# Set the initial velocity to 40
import scipy.optimize as spo
system['speed'] = 40
max_ang = spo.minimize_scalar(height_func,args=system, tol = 0.0001)
print(max_ang.x, max_ang.fun)

The angle that maximizes the height at the wall (about 43 degrees) is a little higher than the angle that maximizes range (which was closer to 40 degrees)  

<br>

We now have the tool to find the angle that optimizes the height at the wall for a given speed.  Now we want to find the lowest speed that will actually clear the wall.  We need a function that takes a speed as an argument and finds the maximum height at the wall for that speed compared to the wall height.  Notice this is a mathematical function:

<br>

$$Height~ above~ wall = f(speed)$$

<br>

As a Python funciton, `speed` will be in the argument, and the "height above the wall" will be our return value.  As it happens, we want the "height above the wall" to be 0, so that it barely clears the wall.  This is convenient, because it means we want the *root* of this function.  So we'll use `root_scalar`!

<br>

Write an `velocity_function` that takes a speed and a `system` object as arguments.  It should use `minimize_scalar` and `height_func` to find the angle that will produce the highest possible height of the ball at the wall, for the given speed (as you did above).  Then it should return the difference between that optimal height and `wall_height`, which is a parameter in `system`.


In [ ]:
# Define the velocity function

def velocity_func(speed,system):
    # Set the system parameter 'speed' to the new value for speed


    # Use `minimize_scalar` to find the angle that gives the maximum
    # height at the wall for this 'speed' (return to 'max_ang')


    # print out the Speed, angle, and maximum height for this speed


    # Return the difference between the top of the wall and the optimal height
    # of the ball at the wall: this is what we want to minimize
    # Remember that max_ang.fun is defined as a negative



In [ ]:
# Test function: should get a result of about 4 m
velocity_func(40,system)

You should get an answer of about 4 m (or possibly -4 m, depending on how you set up the function).  This means that the ball hits 4 m below the top of the wall.

<br>

Now we need find the *root* of the velocity function.  Use `root_scalar` to find the answer to the problem, the minimum speed that gets the ball out of the park. (To refresh yourself on how to use this, look back at notebook 2.4.4):

In [ ]:
# Now call root_scalar to find the angle and minimum speed to clear wall
# Remember that we want a bracket that finds a negative and a positive value
# We know that 40 m/s cannot clear the wall, so it returns a positive value
# Let's guess taht 70 m/s could clear the wall, so it would return a negative value
# Again we're using tolerance to make the simulation
best = spo.root_scalar(velocity_func, system, bracket=[40, 70],xtol=0.001)



In [ ]:
#print(best.root)

Part 4

✅ A.  What is the speed and angle that minimizes the initial speed to clear the fence?  What is the height of the ball for these parameters when it reaches the wall?

✅ ✅ Answer A here.

✅ B.  Look at the print out for speed, angle, and maximum height when `root_scalar` is running.  What is happening with each iteration of `velocity_func`?   Add a similar `print()` statement to `angle_func`, and see if you can determine the process that the simulation is following to come up with an answer.

✅ ✅ Answer B here.